In [ ]:
import vosk
import pyaudio
import json
import pyfirmata
import time

# Initialize connection to Arduino board on COM7 port
board = pyfirmata.Arduino('COM7')

# Load Vosk model for speech recognition (using an Indian English language model)
model = vosk.Model(lang="en-in")

# List of keywords for recognition (helps improve accuracy)
keywords = ["on", "off", "fan", "kitchen", "hall", "bedroom", "light", "terminate"]
rec = vosk.KaldiRecognizer(model, 16000, json.dumps(keywords))

# Initialize PyAudio for real-time audio input
p = pyaudio.PyAudio()

# Define pins for stepper motor control
stepper_pins = [2, 3, 4, 5]  # Arduino digital pins connected to the stepper motor
for pin in stepper_pins:
    board.digital[pin].mode = pyfirmata.OUTPUT  # Set each pin as an OUTPUT

# Define stepper motor step sequence for controlling its rotation
step_sequence = [
    [1, 0, 0, 0],
    [1, 1, 0, 0],
    [0, 1, 0, 0],
    [0, 1, 1, 0],
    [0, 0, 1, 0],
    [0, 0, 1, 1],
    [0, 0, 0, 1],
    [1, 0, 0, 1]
]

# Function to control the stepper motor based on the given direction
def control_stepper_motor(direction, steps=100):
    if direction == "on":
        # Rotate motor in one direction
        for _ in range(steps):
            for step in step_sequence:
                for i in range(4):
                    board.digital[stepper_pins[i]].write(step[i])
                time.sleep(0.01)  # Delay between steps for speed control
    elif direction == "off":
        # Rotate motor in the opposite direction
        for _ in range(steps):
            for step in reversed(step_sequence):
                for i in range(4):
                    board.digital[stepper_pins[i]].write(step[i])
                time.sleep(0.01)  # Delay between steps for speed control

# Dictionary to hold information about registered devices
devices = {}

# Function to register a new device with its associated commands and control pin
def register_device(device_name, pin, on_keywords, off_keywords, is_motor=False):
    """Registers a new device with associated commands and pin."""
    devices[device_name] = {
        "pin": pin,
        "on_keywords": on_keywords,
        "off_keywords": off_keywords,
        "is_motor": is_motor  # Boolean flag to check if it's a motor
    }
    print(f"Device '{device_name}' registered.")

# Registering devices with their respective pins and commands
register_device("hall light", 11, ["on hall"], ["off hall"])
register_device("kitchen light", 13, ["on kitchen"], ["off kitchen"])
register_device("bedroom light", 12, ["on bedroom"], ["off bedroom"])
register_device("fan", stepper_pins, ["on fan", "turn on fan"], ["off fan", "turn off fan"], is_motor=True)

# Path to store recognized speech text output
output_file_path = "recognized_text.txt"

try:
    # Open an audio stream for continuous speech recognition
    stream = p.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=16000,
                    input=True,
                    frames_per_buffer=512)

    # Open the output file in write mode
    with open(output_file_path, "w") as output_file:
        print("Listening for speech. Say 'Terminate' to stop.")

        while True:
            try:
                # Read audio data from the stream
                data = stream.read(512, exception_on_overflow=False)
            except OSError as e:
                print("Input overflowed, skipping this chunk.")
                continue  # Skip current chunk if input overflow occurs

            # Check if Vosk recognizer has a valid result
            if rec.AcceptWaveform(data):
                result = json.loads(rec.Result())
                recognized_text = result['text']  # Extract recognized text

                # Write recognized text to the output file
                output_file.write(recognized_text + "\n")

                # Check if the termination keyword is detected
                if "terminate" in recognized_text.lower():
                    print("Termination keyword detected. Stopping...")
                    break

                # Iterate through each registered device
                for device_name, device_info in devices.items():
                    # Check if any 'ON' command is recognized for the device
                    if any(word in recognized_text.lower() for word in device_info["on_keywords"]):
                        if device_info["is_motor"]:
                            print(f"{device_name} turned ON.")
                            control_stepper_motor("on")  # Activate the stepper motor
                        else:
                            board.digital[device_info["pin"]].write(1)  # Turn on the device
                            print(f"{device_name} turned ON.")
                        time.sleep(1)  # Delay to avoid rapid toggling

                    # Check if any 'OFF' command is recognized for the device
                    elif any(word in recognized_text.lower() for word in device_info["off_keywords"]):
                        if device_info["is_motor"]:
                            print(f"{device_name} turned OFF.")
                            control_stepper_motor("off")  # Deactivate the stepper motor
                        else:
                            board.digital[device_info["pin"]].write(0)  # Turn off the device
                            print(f"{device_name} turned OFF.")
                        time.sleep(1)  # Delay to avoid rapid toggling

    # Stop and close the audio stream
    stream.stop_stream()
    stream.close()

finally:
    # Terminate PyAudio and release the Arduino COM port
    p.terminate()
    board.exit()
    print("Port COM7 has been released.")


Device 'hall light' registered.
Device 'kitchen light' registered.
Device 'bedroom light' registered.
Device 'fan' registered.
Listening for speech. Say 'Terminate' to stop.
Termination keyword detected. Stopping...
Port COM7 has been released.
